Using CNN to Achieve Fruit Classification

In [1]:
import zipfile
import os
zip_file_path = 'fruits-360.zip'
extract_folder = 'fruit'

os.makedirs(extract_folder, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f" {extract_folder}")


文件已解压到: fruit


In [5]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim
import logging
from torch.utils.data import DataLoader
from torchvision.models import resnet34
from timm.data import RandomResizedCropAndInterpolation
from torchvision.transforms import RandomErasing,RandAugment

logging.basicConfig(
    filename="log34-40.txt",  
    filemode="w", 
    level=logging.INFO, 
    format="%(asctime)s - %(message)s",  
    datefmt="%Y-%m-%d %H:%M:%S",
)


def log_print(message):
    print(message) 
    logging.info(message) 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
log_print(f"Using device: {device}")

train_transform = transforms.Compose([
    RandomResizedCropAndInterpolation(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation='bicubic'),
    transforms.RandomHorizontalFlip(p=0.5),
    RandAugment(num_ops=2,magnitude=9),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    RandomErasing(p=0.5) 
])

val_transform = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC, antialias=True),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder(root="fruits-360/5%/train", transform=train_transform)
val_dataset = datasets.ImageFolder(root="fruits-360/5%/val", transform=val_transform)

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

num_classes = len(train_dataset.classes)
log_print(f" include {num_classes} category")

model = resnet34(pretrained=False)
model.fc = nn.Linear(512, num_classes)  
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=2.5e-4, weight_decay=0.05)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

def train(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=30):
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        scheduler.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        val_acc = correct / total
        log_print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {running_loss/len(train_loader):.4f}, "
                  f"Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "resnet18_fruits360.pth")
            log_print("Model Saved!")

log_print("train ResNet-34 ...")
train(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=20)
log_print("finish！")


Using device: cuda
 数据集中包含 141 种水果类别
开始训练 ResNet-34 ...
Epoch [1/20] - Loss: 4.6127, Train Acc: 0.0533, Val Acc: 0.1295
Model Saved!
Epoch [2/20] - Loss: 3.5520, Train Acc: 0.1479, Val Acc: 0.2836
Model Saved!
Epoch [3/20] - Loss: 3.2178, Train Acc: 0.1870, Val Acc: 0.3289
Model Saved!
Epoch [4/20] - Loss: 3.0317, Train Acc: 0.2205, Val Acc: 0.3271
Epoch [5/20] - Loss: 2.9530, Train Acc: 0.2278, Val Acc: 0.3008
Epoch [6/20] - Loss: 2.8597, Train Acc: 0.2454, Val Acc: 0.2499
Epoch [7/20] - Loss: 2.7990, Train Acc: 0.2602, Val Acc: 0.3531
Model Saved!
Epoch [8/20] - Loss: 2.6779, Train Acc: 0.2735, Val Acc: 0.4014
Model Saved!
Epoch [9/20] - Loss: 2.6844, Train Acc: 0.2860, Val Acc: 0.4127
Model Saved!
Epoch [10/20] - Loss: 2.7507, Train Acc: 0.2724, Val Acc: 0.3390
Epoch [11/20] - Loss: 2.3971, Train Acc: 0.3514, Val Acc: 0.7128
Model Saved!
Epoch [12/20] - Loss: 2.1308, Train Acc: 0.4512, Val Acc: 0.7821
Model Saved!
Epoch [13/20] - Loss: 2.0727, Train Acc: 0.4605, Val Acc: 0.7772
Epoc